### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Please see the summary of trends in the word document
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

## Player Count

* Display the total number of players


In [2]:
Total_players=len(purchase_data["SN"].unique())
summary_count=pd.DataFrame({"Total Players":[Total_players]})
summary_count

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [3]:
# calculate
Number_of_unique_items=len(purchase_data["Item Name"].unique())
Average_price=purchase_data["Price"].mean()
Numbers_of_purchase=len(purchase_data["Purchase ID"])
Total_Revenue=purchase_data["Price"].sum()
# Generate a table
summary_purchase=pd.DataFrame({"Number of Unique Items":[Number_of_unique_items],
                                "Average Price":[Average_price],
                                "Number of Purchases":[Numbers_of_purchase],
                                 "Total Revenue":[Total_Revenue]})
# format the price and revenue with dollar sign
summary_purchase["Average Price"]=summary_purchase["Average Price"].map("${:,.2f}".format)
summary_purchase["Total Revenue"]=summary_purchase["Total Revenue"].map("${:,.2f}".format)
# print out the summary table
summary_purchase

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [4]:
#calculate and remove duplicates in SN
Gender=purchase_data[['SN', 'Gender']]
Gender_data=Gender.drop_duplicates()
Gender_count=Gender_data['Gender'].value_counts()

Total=Gender_count.sum()

Percentage_male=Gender_count[0]/Total*100
Percentage_female=Gender_count[1]/Total*100
Percentage_others=Gender_count[2]/Total*100

#Generate summary table
summary_demographics=pd.DataFrame({"Total count":[Gender_count[0], Gender_count[1], Gender_count[2]],
     "Percentage of Players":[Percentage_male, Percentage_female,Percentage_others]})
summary_demographics.index=(["Male", "Female", "Others/Non-Disclosed"])

#format the percentages
summary_demographics["Percentage of Players"]=summary_demographics["Percentage of Players"].map("{:.2f}%".format)
summary_demographics

,Total count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Others/Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [5]:
# Groupby gender and do calculation
Groupbygender=purchase_data.groupby(["Gender"])
Purchase_count=Groupbygender["SN"].count()
Average_purchase_price=Groupbygender["Price"].mean()
Total_purchase_value=Groupbygender["Price"].sum()
Average_total_purchase_per_person=Total_purchase_value/Gender_count
# Generate a summary table
summary_purchasing_gender=pd.DataFrame({"Purchase Count":Purchase_count,
                                       "Average Purchase Price":Average_purchase_price,
                                       "Total Purchase Value":Total_purchase_value, 
                                    "Avg Total Purchase per Person":Average_total_purchase_per_person})
# format price, vaule and avg total price
summary_purchasing_gender=summary_purchasing_gender.style.format({'Average Purchase Price':'${:,.2f}',
                                       'Total Purchase Value':'${:,.2f}',
                                      'Avg Total Purchase per Person':'${:,.2f}'})
# print out
summary_purchasing_gender

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [6]:
# Bin the data based on ages
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200]
group_names=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data["Age Demographics"] = pd.cut(purchase_data["Age"], bins, labels=group_names, include_lowest=True)

# add an extra column of "Age Deomographics"
Age_Demographics = purchase_data.groupby("Age Demographics")

# find the unique SN
Count_df = Age_Demographics.agg({"SN": "nunique"})

# calculate
Total=Count_df.sum()
Percentage_of_players= round((Count_df/Total*100), 2)

Count_df=Count_df.reset_index()
Percentage_of_players=Percentage_of_players.reset_index()

Summary_age = Count_df.merge(Percentage_of_players, on="Age Demographics")
# Change the index to Age Demographics
Summary_age.set_index("Age Demographics", inplace=True)

#replace the column headers
Summary_age=Summary_age.rename(columns={"SN_x":"Total Count",
                                        "SN_y": "Percentage of Players"})
#Drop the index name
Summary_age.index.name = None
#fomart the percentage
Summary_age["Percentage of Players"]=Summary_age["Percentage of Players"].map("{:.2f}%".format)
#print out summary table
Summary_age

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [7]:
# calculate
pc=Age_Demographics["Purchase ID"].count()
ap=Age_Demographics["Price"].mean()
tpv=Age_Demographics["Price"].sum()

#Set index of Age demographics 
ct=Count_df.set_index("Age Demographics")

#Merge two output dataframe together
ct_2=ct.merge(tpv, on="Age Demographics")

# Calculate Avg Total Purchase per Person (tppp)
tppp=ct_2["Price"]/ct_2["SN"]

# Generate a summary table
summary_purchasing_age=pd.DataFrame({"Purchase Count":pc,
                                       "Average Purchase Price":ap,
                                       "Total Purchase Value":tpv, 
                                    "Avg Total Purchase per Person":tppp})
# format price, vaule and avg total price
summary_purchasing_age=summary_purchasing_age.style.format({'Average Purchase Price':'${:,.2f}',
                                       'Total Purchase Value':'${:,.2f}',
                                      'Avg Total Purchase per Person':'${:,.2f}'})

# print
summary_purchasing_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Demographics,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [8]:
# Groupby by both SN and get the Price column out
Spender=purchase_data.groupby("SN")["Price"]
# calculate
Spender_count = Spender.count()
Spender_mean=Spender.mean()
Spender_sum=Spender.sum()
# generate a summary table
Spender_df=pd.DataFrame({'Purchase Count':Spender_count,
                        'Average Purchase Price': Spender_mean,
                        'Total Purchase Value': Spender_sum})
# sort based on the Total purchase value by descending order
Spender_sort=Spender_df.sort_values('Total Purchase Value', ascending=False)

#display the first five
Spender_display=Spender_sort.head(5)

#format 
Spender_display=Spender_display.style.format({'Average Purchase Price':'${:,.2f}',
                                     'Total Purchase Value':'${:,.2f}'})
Spender_display

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Generate a new dataframe with only three columns
most_popular_items=purchase_data[['Item ID', 'Item Name', 'Price']]
# Groupby two variables: item ID and item name
mpi_groupby=most_popular_items.groupby(['Item ID','Item Name'])
# Calculate
purchase_count=mpi_groupby['Item Name'].count()
Total_purchase=mpi_groupby['Price'].sum()
Item_price=Total_purchase/purchase_count
#generate the summary table
mpi=pd.DataFrame({"Purchase Count":purchase_count,
                 "Item Price": Item_price,
                 "Total Purchase Value":Total_purchase})
# sort values based on the purchase count
mpi_sort=mpi.sort_values("Purchase Count",ascending=False)
# display only five and format
mpi_display=mpi_sort.head(5)
mpi_display=mpi_display.style.format({'Item Price':'${:,.2f}',
                                     'Total Purchase Value':'${:,.2f}'})
mpi_display

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [10]:
# use the above dataframe as input and sort based on total purchase value
mpi_sort_2=mpi.sort_values("Total Purchase Value",ascending=False)
# display the first five and format
mpi_sort_2_display=mpi_sort_2.head(5)
mpi_sort_2_display=mpi_sort_2_display.style.format({'Item Price':'${:,.2f}',
                                     'Total Purchase Value':'${:,.2f}'})
mpi_sort_2_display

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
